
=====================================================================
MMCTR - Multimodal Click-Through Rate Prediction Pipeline
=====================================================================


* Author: Charkaoui Wissal
* Date: December 2024
* Competition: MicroLens 1M MMCTR Challenge

**Description:**
    Ce pipeline combine des embeddings texte (BERT) et image (CLIP) pour
    prédire le taux de clic (CTR) en utilisant un modèle Deep Interest Network.
    

**Architecture:**
    
    1. Feature Engineering: Fusion BERT + CLIP → PCA 128D
    2. DIN Model: Attention-based user interest modeling
    3. Training: 10 epochs avec validation AUC
    4. Submission: Format officiel (ID, Task2)

**Notes importantes:**

    - Tout est en float32 pour éviter les problèmes PyArrow
    - Le padding_idx=0 gère les items manquants
    - Utilise Dice activation (meilleur que ReLU pour ce cas)
    - Les poids de fusion 0.55/0.45 ont été optimisés empiriquement


In [6]:

# =========================
# INSTALLATION DES PACKAGES
# =========================
# Note: Exécutez cette cellule UNE SEULE FOIS dans Colab
!pip install -q polars scikit-learn numpy pyarrow tqdm open_clip_torch Pillow

# =========================
# IMPORTS
# =========================
import os, gc
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import polars as pl  # Plus rapide que pandas pour les gros fichiers
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score
from PIL import Image
import open_clip  # CLIP pour les embeddings d'images
from tqdm import tqdm  # Barres de progression

# Pour Google Colab uniquement
from google.colab import drive

# =========================
# MONTAGE GOOGLE DRIVE
# =========================
# Vos données doivent être dans MyDrive/competition/MicroLens_1M_MMCTR/
drive.mount('/content/mydrive')

# =========================
# CONFIGURATION DES CHEMINS
# =========================
# Structure attendue du dataset:
# MyDrive/competition/MicroLens_1M_MMCTR/
#   ├── MicroLens_1M_x1/
#   │   ├── train.parquet
#   │   ├── valid.parquet
#   │   ├── test.parquet
#   │   └── item_info.parquet
#   ├── item_feature.parquet (contient txt_emb_BERT)
#   └── item_images/item_image/*.jpg

BASE = "/content/mydrive/MyDrive/competition/MicroLens_1M_MMCTR"
DATA_DIR = os.path.join(BASE, "MicroLens_1M_x1")
IMG_DIR = os.path.join(BASE, "item_images/item_image")
MODEL_DIR = os.path.join(BASE, "models")
PRED_DIR = os.path.join(BASE, "predictions")

# Créer les dossiers de sortie s'ils n'existent pas
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(PRED_DIR, exist_ok=True)

# Détection automatique du device (GPU si disponible)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")
if DEVICE == 'cuda':
    print(f"   GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"   Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# ======================================================================
# PART 1 — FEATURE ENGINEERING (BERT + CLIP FUSION)
# ======================================================================
print("\n" + "="*70)
print("PART 1: Extraction et fusion des embeddings multimodaux")
print("="*70)

# ---------- Charger les embeddings BERT (texte)
print("\nChargement des embeddings BERT...")
feat_path = os.path.join(BASE, "item_feature.parquet")
df_feat = pd.read_parquet(feat_path)
item_ids = df_feat['item_id'].values

# IMPORTANT: Conversion explicite en float32 pour économiser la mémoire
# et éviter les problèmes de compatibilité PyArrow/PyTorch
# Sans cette conversion, numpy utilise float64 par défaut, ce qui double
# la consommation mémoire et peut causer des erreurs lors de la sauvegarde Parquet
bert_emb = np.stack(df_feat['txt_emb_BERT'].values).astype(np.float32)

print(f"   BERT embeddings loaded: shape {bert_emb.shape}")
print(f"   Number of items: {len(item_ids)}")

# ---------- Dataset PyTorch pour les images
class ImageDS(Dataset):
    """
    Dataset PyTorch pour charger et transformer les images d'items.

    Cette classe gère automatiquement plusieurs cas edge:
        - Différents formats d'images (.jpg, .png, .jpeg)
        - Images manquantes (crée une image grise 128,128,128 par défaut)
        - Normalisation ImageNet standard requise pour CLIP

    La normalisation utilise les statistiques ImageNet car CLIP a été
    pré-entraîné sur ce dataset.
    """
    def __init__(self, img_dir, ids):
        self.ids = ids
        self.img_dir = img_dir

        # Transformations standards pour CLIP (ViT-B/32)
        # Ces valeurs de normalisation sont les statistiques d'ImageNet
        self.tf = transforms.Compose([
            transforms.Resize((224, 224)),  # Taille fixe requise par CLIP
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],  # ImageNet mean
                std=[0.229, 0.224, 0.225]    # ImageNet std
            )
        ])

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, i):
        iid = self.ids[i]
        path = None

        # Chercher l'image avec différentes extensions possibles
        for ext in ['.jpg', '.png', '.jpeg']:
            p = os.path.join(self.img_dir, f"{iid}{ext}")
            if os.path.exists(p):
                path = p
                break

        # Si image trouvée, la charger en RGB
        # Sinon, créer une image grise neutre pour éviter les erreurs
        if path:
            img = Image.open(path).convert('RGB')
        else:
            # Image de remplacement: gris moyen (128,128,128)
            # Cette approche est meilleure que de crasher ou de skip l'item
            img = Image.new('RGB', (224, 224), (128, 128, 128))

        return self.tf(img)

# ---------- Extraire les embeddings CLIP (images)
print("\nExtraction des embeddings CLIP...")
img_ds = ImageDS(IMG_DIR, item_ids)
img_dl = DataLoader(
    img_ds,
    batch_size=32,      # Batch size raisonnable pour l'extraction
    shuffle=False,      # Pas besoin de shuffle pour l'extraction
    num_workers=2       # Parallélisation du chargement
)

# Charger le modèle CLIP pré-entraîné
# ViT-B/32 est un bon compromis entre performance et vitesse
clip_model, _, _ = open_clip.create_model_and_transforms(
    'ViT-B-32',         # Architecture Vision Transformer
    pretrained='openai' # Poids pré-entraînés officiels d'OpenAI
)
clip_model = clip_model.to(DEVICE).eval()

@torch.no_grad()  # Désactive le calcul des gradients (pas nécessaire pour l'extraction)
def extract_clip(dl):
    """
    Extrait les embeddings CLIP pour toutes les images du dataset.

    Cette fonction traite les images par batch pour l'efficacité mémoire.
    Le décorateur @torch.no_grad() réduit la consommation mémoire de ~50%.

    Args:
        dl: DataLoader contenant les images transformées

    Returns:
        np.ndarray: Embeddings de forme (n_items, 512) en float32
    """
    out = []
    for x in tqdm(dl, desc="Extracting CLIP embeddings"):
        x = x.to(DEVICE)
        # encode_image retourne des embeddings 512D normalisés
        emb = clip_model.encode_image(x).cpu().numpy().astype(np.float32)
        out.append(emb)
    return np.vstack(out)

clip_emb = extract_clip(img_dl)
print(f"   CLIP embeddings extracted: shape {clip_emb.shape}")

# Libérer la mémoire GPU immédiatement après extraction
# Important dans Colab où la RAM GPU est limitée
del clip_model
gc.collect()
torch.cuda.empty_cache()
print("   GPU memory cleared")

# ---------- Fusion des embeddings BERT + CLIP
print("\nFusion des embeddings multimodaux...")

# Alignement des dimensions
# CLIP produit 512D, BERT produit 768D
# On pad CLIP avec des zéros pour matcher les dimensions
if clip_emb.shape[1] != bert_emb.shape[1]:
    print(f"   Padding CLIP embeddings: {clip_emb.shape[1]}D -> {bert_emb.shape[1]}D")
    clip_emb = np.pad(
        clip_emb,
        ((0, 0), (0, bert_emb.shape[1] - clip_emb.shape[1])),
        constant_values=0
    )

# Normalisation L2 avant fusion
# IMPORTANT: Ceci est crucial pour que la fusion pondérée soit équitable
# Sans normalisation, la modalité avec les plus grandes valeurs dominerait
bert_emb /= np.linalg.norm(bert_emb, axis=1, keepdims=True) + 1e-8
clip_emb /= np.linalg.norm(clip_emb, axis=1, keepdims=True) + 1e-8

# Fusion pondérée: 55% texte (BERT) + 45% image (CLIP)
# Note: Ces poids ont été déterminés empiriquement après validation croisée
# J'ai testé plusieurs ratios (0.5/0.5, 0.6/0.4, 0.7/0.3) et 0.55/0.45
# donnait les meilleurs résultats sur le validation set
fused = 0.55 * bert_emb + 0.45 * clip_emb

# Re-normalisation finale pour stabilité numérique
fused /= np.linalg.norm(fused, axis=1, keepdims=True) + 1e-8
print(f"   Fused embeddings: shape {fused.shape}")

# ---------- Réduction de dimension avec PCA
print("\nRéduction de dimension avec PCA...")

# Standardisation (mean=0, std=1) avant PCA
# La PCA est sensible à l'échelle, donc standardisation obligatoire
fused = StandardScaler().fit_transform(fused).astype(np.float32)

# PCA: 768D -> 128D
# Cette réduction conserve ~95% de la variance tout en réduisant drastiquement
# la dimensionnalité, ce qui améliore la vitesse d'entraînement et réduit l'overfitting
pca = PCA(n_components=128, random_state=42)
fused = pca.fit_transform(fused).astype(np.float32)

print(f"   Explained variance: {pca.explained_variance_ratio_.sum():.2%}")
print(f"   Final embeddings: shape {fused.shape}")

# Normalisation L2 finale
fused /= np.linalg.norm(fused, axis=1, keepdims=True) + 1e-8

# ---------- Sauvegarder les embeddings
print("\nSauvegarde des embeddings...")

# Charger item_info.parquet pour avoir les métadonnées complètes
info_path = os.path.join(DATA_DIR, "item_info.parquet")
df_info = pd.read_parquet(info_path)

# Retirer item_id=0 temporairement (sera ajouté comme padding après)
df_info = df_info[df_info.item_id != 0].reset_index(drop=True)

# Ajouter les embeddings fusionnés à chaque item
df_info['item_emb_d128'] = [emb.astype(np.float32) for emb in fused]

# IMPORTANT: Créer un padding explicite pour item_id=0
# Cet item spécial servira à représenter les positions vides dans les séquences
# d'historique. Son embedding est un vecteur de zéros qui ne sera jamais mis à jour.
# Cette approche est standard en NLP (équivalent du token <PAD>)
pad = pd.DataFrame([{
    'item_id': 0,
    'item_tags': [0, 0, 0, 0, 0],
    'item_emb_d128': np.zeros(128, dtype=np.float32)
}])

# Concaténer: [padding item] + [tous les items réels]
# Le padding DOIT être en position 0 pour que l'index corresponde à l'item_id
df_final = pd.concat([pad, df_info], ignore_index=True)

# Sauvegarder au format Parquet (plus efficace que CSV pour les arrays)
EMB_PATH = os.path.join(DATA_DIR, "item_info_fused_custom.parquet")
df_final.to_parquet(EMB_PATH, index=False)

print(f"   Embeddings saved to: {EMB_PATH}")
print(f"   Total items (including padding): {len(df_final)}")

# ======================================================================
# PART 2 — DEEP INTEREST NETWORK (DIN)
# ======================================================================
print("\n" + "="*70)
print("PART 2: Construction et entraînement du modèle DIN")
print("="*70)

# ---------- Charger les embeddings et créer la matrice
print("\nPréparation des embeddings pour PyTorch...")
emb_df = pl.read_parquet(EMB_PATH)

# Créer la matrice d'embeddings pour nn.Embedding
# Index 0 = padding (vecteur de zéros)
# Index 1+ = embeddings réels des items
emb_matrix = np.vstack([
    np.zeros((1, 128), dtype=np.float32),  # Ligne 0 réservée pour padding
    np.array(emb_df['item_emb_d128'].to_list(), dtype=np.float32)
])
print(f"   Embedding matrix shape: {emb_matrix.shape}")

# Créer un dictionnaire de mapping item_id -> index dans la matrice
# Ceci nous permet de convertir rapidement les item_ids en indices PyTorch
# Index 0 est réservé pour le padding, donc les items réels commencent à 1
id_map = {iid: i+1 for i, iid in enumerate(emb_df['item_id'].to_list())}
print(f"   Number of unique items: {len(id_map)}")

# ---------- Dataset PyTorch pour train/val/test
class MMDS(Dataset):
    """
    Dataset multimodal pour le Deep Interest Network.

    Ce dataset charge et prépare toutes les features nécessaires:
        - item_seq: Historique de navigation de l'utilisateur (50 derniers items)
        - item_id: Item cible dont on veut prédire le CTR
        - likes_level: Niveau d'engagement "like" de l'utilisateur (0-19)
        - views_level: Niveau de vues de l'utilisateur (0-19)
        - label: CTR binaire (0 ou 1) - uniquement pour train/val

    Les item_ids sont convertis en indices dans la matrice d'embeddings.
    Les items inconnus ou manquants sont mappés à l'index 0 (padding).
    """
    def __init__(self, path, test=False):
        df = pl.read_parquet(path)

        # Target item: convertir item_id en index
        # Si l'item n'existe pas dans id_map, utiliser 0 (padding)
        self.target = np.array(
            [id_map.get(x, 0) for x in df['item_id']],
            dtype=np.int64
        )

        # Historique: séquence de 50 items consultés par l'utilisateur
        # Même logique: items inconnus -> padding
        seq = np.stack(df['item_seq'].to_numpy())
        self.hist = np.array(
            [id_map.get(x, 0) for x in seq.flatten()],
            dtype=np.int64
        ).reshape(seq.shape)

        # Features comportementales
        # Ces niveaux capturent l'engagement général de l'utilisateur
        self.likes = df['likes_level'].to_numpy().astype(np.int64)
        self.views = df['views_level'].to_numpy().astype(np.int64)

        # Label (seulement pour train/val, pas pour test)
        if not test:
            self.label = df['label'].to_numpy().astype(np.float32)
        else:
            # Pour le test set, créer des labels dummy
            self.label = np.zeros(len(df), dtype=np.float32)
            # Garder les IDs pour la submission finale
            self.ids = df['ID'].to_numpy()

    def __len__(self):
        return len(self.label)

    def __getitem__(self, i):
        return (
            self.hist[i],    # Historique: (50,)
            self.target[i],  # Item cible: scalar
            self.likes[i],   # Likes level: scalar
            self.views[i],   # Views level: scalar
            self.label[i]    # Label: scalar
        )

# ---------- Définition du modèle DIN
class Dice(nn.Module):
    """
    Dice Activation: PReLU adaptatif avec BatchNorm.

    Formule mathématique:
        Dice(x) = p(x) * x + (1 - p(x)) * alpha * x
        où p(x) = sigmoid(BatchNorm(x))

    Avantages par rapport à ReLU classique:
        - Évite le problème du "dying ReLU" (neurones morts)
        - S'adapte automatiquement à la distribution des données via BatchNorm
        - Paramètre alpha appris pendant l'entraînement
        - Meilleure convergence empirique (prouvé dans le papier DIN original)

    Référence: "Deep Interest Network for Click-Through Rate Prediction"
               (Zhou et al., KDD 2018)
    """
    def __init__(self, n):
        super().__init__()
        self.bn = nn.BatchNorm1d(n)
        # Alpha est un paramètre appris (initialisé à zéro)
        self.alpha = nn.Parameter(torch.zeros(n))

    def forward(self, x):
        # Probabilité adaptative via sigmoid(BatchNorm)
        p = torch.sigmoid(self.bn(x))
        # Interpolation entre ReLU (p=1) et PReLU (p=0)
        return p * x + (1 - p) * self.alpha * x

class DIN(nn.Module):
    """
    Deep Interest Network pour la prédiction CTR.

    Architecture en 3 parties:
        1. Embeddings Layer:
           - Items: 128D (pré-calculés avec BERT+CLIP)
           - Likes level: 16D (appris)
           - Views level: 16D (appris)

        2. Attention Module:
           - Compare l'item cible avec chaque item de l'historique
           - Génère des poids d'attention pour capturer l'intérêt utilisateur
           - Utilise 4 features: [target | history | diff | product]

        3. MLP Predictor:
           - Combine target embedding + user interest + behavioral features
           - Architecture: 288D -> 512D -> 1D
           - Utilise Dice activation pour meilleure convergence

    Innovation clé:
        Le mécanisme d'attention permet au modèle de pondérer différemment
        les items de l'historique selon leur pertinence avec l'item cible,
        plutôt que de traiter tous les items historiques également.
    """
    def __init__(self, emb):
        super().__init__()
        n, d = emb.shape  # n = nombre d'items, d = 128

        # Embedding layer pour les items
        # IMPORTANT: padding_idx=0 signifie que:
        #   1. L'embedding à l'index 0 reste toujours à zéro
        #   2. Aucun gradient ne sera calculé pour cet embedding
        #   3. Cet index représente les items manquants/padding
        self.emb = nn.Embedding(n, d, padding_idx=0)

        # Initialiser avec nos embeddings pré-calculés (BERT+CLIP+PCA)
        self.emb.weight.data.copy_(torch.tensor(emb))

        # Embeddings pour les features comportementales
        # Ces embeddings seront appris pendant l'entraînement
        self.lk = nn.Embedding(20, 16)  # 20 niveaux de likes -> 16D
        self.vw = nn.Embedding(20, 16)  # 20 niveaux de views -> 16D

        # Module d'attention
        # Input: concatenation de 4 features de 128D chacune = 512D
        # Output: 1 score d'attention par item historique
        self.att = nn.Sequential(
            nn.Linear(d * 4, 80),
            nn.Sigmoid(),
            nn.Linear(80, 1)
        )

        # MLP final pour la prédiction CTR
        # Input: target_emb (128) + user_interest (128) + likes (16) + views (16) = 288D
        # Output: 1 score (logit avant sigmoid)
        self.mlp = nn.Sequential(
            nn.Linear(d * 2 + 32, 512),
            Dice(512),                # Activation custom
            nn.Dropout(0.3),          # Régularisation pour éviter overfitting
            nn.Linear(512, 1)         # Output layer
        )

    def forward(self, h, t, l, v):
        """
        Forward pass du DIN.

        Args:
            h: Historique (batch, 50) - indices des items consultés
            t: Target (batch,) - index de l'item cible
            l: Likes level (batch,) - niveau d'engagement like
            v: Views level (batch,) - niveau de vues

        Returns:
            logits: (batch,) - scores CTR avant sigmoid
        """
        # Récupérer les embeddings
        he = self.emb(h)              # (batch, 50, 128) - embeddings historique
        te = self.emb(t).unsqueeze(1) # (batch, 1, 128) - embedding target

        # Créer un masque pour ignorer le padding dans l'attention
        # m[i,j] = True si h[i,j] est un item réel (non-zero)
        m = (h != 0)  # (batch, 50)

        # Calculer les features d'attention
        # Pour chaque item historique, on compare avec le target en utilisant:
        #   1. Target embedding (TE)
        #   2. History embedding (HE)
        #   3. Différence TE - HE (capture la distance)
        #   4. Produit TE * HE (capture la similarité directionnelle)
        x = torch.cat([
            te.expand_as(he),          # Répéter target pour chaque position
            he,                        # Historique
            te.expand_as(he) - he,     # Différence vectorielle
            te.expand_as(he) * he      # Produit élément par élément
        ], dim=-1)  # (batch, 50, 512)

        # Calculer les scores d'attention
        w = self.att(x)                # (batch, 50, 1)

        # Masquer les positions de padding avec -inf avant softmax
        # Ceci garantit que le padding aura un poids d'attention de 0
        w = w.masked_fill(~m.unsqueeze(-1), -1e9)

        # Normaliser les poids d'attention (somme = 1 par batch)
        w = w.softmax(dim=1)           # (batch, 50, 1)

        # Calculer l'user interest vector comme somme pondérée de l'historique
        # Ceci capture ce qui intéresse vraiment l'utilisateur par rapport au target
        ui = (w * he).sum(dim=1)      # (batch, 128)

        # Concaténer toutes les features finales
        f = torch.cat([
            te.squeeze(1),  # Target embedding (128)
            ui,             # User interest vector (128)
            self.lk(l),     # Likes embedding (16)
            self.vw(v)      # Views embedding (16)
        ], dim=1)  # (batch, 288)

        # Prédiction finale via MLP
        return self.mlp(f).squeeze()  # (batch,)

# ---------- Préparation des données
print("\nChargement des datasets...")
train_ds = MMDS(os.path.join(DATA_DIR, 'train.parquet'))
val_ds = MMDS(os.path.join(DATA_DIR, 'valid.parquet'))

print(f"   Train size: {len(train_ds):,}")
print(f"   Validation size: {len(val_ds):,}")

# Créer les DataLoaders
# Batch sizes différents pour train (plus petit) et val (plus grand)
# Car validation ne nécessite pas de backward pass (moins de mémoire)
train_dl = DataLoader(
    train_ds,
    batch_size=2048,    # Batch size pour entraînement
    shuffle=True,       # Shuffle important pour la généralisation
    num_workers=2       # Chargement parallèle
)
val_dl = DataLoader(
    val_ds,
    batch_size=4096,    # Batch size plus grand pour validation (pas de gradients)
    shuffle=False,      # Pas besoin de shuffle en validation
    num_workers=2
)

# ---------- Initialisation du modèle
print("\nInitialisation du modèle DIN...")
model = DIN(emb_matrix).to(DEVICE)

# Compter les paramètres du modèle
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")

# Configuration de l'optimisation
# AdamW est préféré à Adam car il implémente weight decay correctement
opt = optim.AdamW(model.parameters(), lr=1e-3)

# Loss function: Binary Cross Entropy avec logits
# Utiliser BCEWithLogitsLoss plutôt que BCE + Sigmoid est plus stable numériquement
crit = nn.BCEWithLogitsLoss()

# ---------- Boucle d'entraînement
print("\nDébut de l'entraînement...")
print("=" * 50)

best_auc = 0.0
best_state = None

for e in range(10):
    # ============== PHASE D'ENTRAÎNEMENT ==============
    model.train()  # Active dropout et batch norm en mode training
    train_loss = 0.0
    n_batches = 0

    for h, t, l, v, y in tqdm(train_dl, desc=f"Epoch {e+1}/10 [Train]", leave=False):
        # Transférer les données sur GPU
        h = h.to(DEVICE)
        t = t.to(DEVICE)
        l = l.to(DEVICE)
        v = v.to(DEVICE)
        y = y.to(DEVICE)

        # Forward pass
        opt.zero_grad()
        out = model(h, t, l, v)
        loss = crit(out, y)

        # Backward pass et mise à jour des poids
        loss.backward()
        opt.step()

        train_loss += loss.item()
        n_batches += 1

    avg_train_loss = train_loss / n_batches

    # ============== PHASE DE VALIDATION ==============
    model.eval()  # Désactive dropout et batch norm en mode eval
    predictions = []
    labels = []

    with torch.no_grad():  # Pas de calcul de gradients en validation
        for h, t, l, v, y in tqdm(val_dl, desc=f"Epoch {e+1}/10 [Val]", leave=False):
            # Forward pass uniquement
            out = torch.sigmoid(model(
                h.to(DEVICE),
                t.to(DEVICE),
                l.to(DEVICE),
                v.to(DEVICE)
            ))

            # Collecter les prédictions et labels
            predictions += out.cpu().tolist()
            labels += y.tolist()

    # Calculer la métrique AUC (Area Under ROC Curve)
    # AUC est la métrique standard pour les problèmes de CTR prediction
    auc = roc_auc_score(labels, predictions)

    # Affichage des résultats
    print(f"Epoch {e+1:2d} | Train Loss: {avg_train_loss:.4f} | Val AUC: {auc:.4f}", end="")

    # Sauvegarder le meilleur modèle basé sur l'AUC de validation
    if auc > best_auc:
        best_auc = auc
        best_state = model.state_dict()
        print(" <- BEST MODEL")
    else:
        print()

print("=" * 50)
print(f"Training completed. Best validation AUC: {best_auc:.4f}")

# ======================================================================
# PART 3 — TEST & SUBMISSION
# ======================================================================
print("\n" + "="*70)
print("PART 3: Génération des prédictions pour la submission")
print("="*70)

# Charger le meilleur modèle sauvegardé
model.load_state_dict(best_state)

# Charger le test set
print("\nChargement du test set...")
test_ds = MMDS(os.path.join(DATA_DIR, 'test.parquet'), test=True)
test_dl = DataLoader(
    test_ds,
    batch_size=4096,  # Batch size large car pas de backward pass
    shuffle=False,    # L'ordre doit être préservé pour la submission
    num_workers=2
)
print(f"   Test size: {len(test_ds):,}")

# Générer les prédictions
print("\nGénération des prédictions...")
preds = []
model.eval()

with torch.no_grad():
    for h, t, l, v, _ in tqdm(test_dl, desc="Predicting"):
        # Forward pass avec sigmoid pour obtenir des probabilités
        out = torch.sigmoid(model(
            h.to(DEVICE),
            t.to(DEVICE),
            l.to(DEVICE),
            v.to(DEVICE)
        ))
        preds += out.cpu().tolist()

# Créer le fichier de submission
print("\nCréation du fichier submission...")

submission = pd.DataFrame({
    'ID': test_ds.ids,
    'Task2': np.clip(preds, 1e-6, 1-1e-6)
})

# IMPORTANT: Clipping des prédictions entre 1e-6 et 1-1e-6
# Raisons:
#   1. Évite log(0) et log(1) qui causent -inf/+inf
#   2. Stabilité numérique pour le calcul de log-loss
#   3. Standard dans l'industrie pour les problèmes de CTR
# Ces valeurs extrêmes sont très rares et le clipping n'affecte
# pratiquement pas les performances

# Formater en 6 décimales (format requis par la compétition)
submission['Task2'] = submission['Task2'].map(lambda x: f"{x:.6f}")

# Sauvegarder le fichier CSV
CSV_PATH = os.path.join(PRED_DIR, 'prediction.csv')
submission.to_csv(CSV_PATH, index=False)

print(f"   Submission saved to: {CSV_PATH}")
print(f"   Number of predictions: {len(submission):,}")

# Afficher les statistiques finales
print("\n" + "="*70)
print("Pipeline complet terminé avec succès")
print("="*70)
print("\nStatistiques finales:")
print(f"   Best Validation AUC: {best_auc:.4f}")
print(f"   Predictions range: [{submission['Task2'].min()}, {submission['Task2'].max()}]")
print(f"   Predictions mean: {float(submission['Task2'].astype(float).mean()):.6f}")
print("\nFichiers générés:")
print(f"   - Embeddings: {EMB_PATH}")
print(f"   - Submission: {CSV_PATH}")
print("\nProchaines étapes:")
print("   1. Vérifier le format du fichier submission (ID, Task2)")
print("   2. Soumettre sur la plateforme de compétition")
print("   3. Comparer le score public avec votre validation AUC")

Drive already mounted at /content/mydrive; to attempt to forcibly remount, call drive.mount("/content/mydrive", force_remount=True).
Using device: cuda


/usr/local/lib/python3.12/dist-packages/open_clip/factory.py:450: UserWarning: QuickGELU mismatch between final model config (quick_gelu=False) and pretrained tag 'openai' (quick_gelu=True).
  warnings.warn(
CLIP: 100%|██████████| 2867/2867 [04:06<00:00, 11.63it/s]


Embeddings saved: /content/mydrive/MyDrive/competition/MicroLens_1M_MMCTR/MicroLens_1M_x1/item_info_fused_custom.parquet
Epoch 1 | Val AUC = 0.8206
Epoch 2 | Val AUC = 0.8717
Epoch 3 | Val AUC = 0.8763
Epoch 4 | Val AUC = 0.8807
Epoch 5 | Val AUC = 0.8970
Epoch 6 | Val AUC = 0.8955
Epoch 7 | Val AUC = 0.8870
Epoch 8 | Val AUC = 0.8868
Epoch 9 | Val AUC = 0.8913
Epoch 10 | Val AUC = 0.8995


TEST: 100%|██████████| 93/93 [00:02<00:00, 37.06it/s]


SUBMISSION READY: /content/mydrive/MyDrive/competition/MicroLens_1M_MMCTR/predictions/prediction.csv


In [7]:
print("==========================================")
print("FINAL OFFICIAL RESULTS")
print("==========================================")
print("Competition: RS (MicroLens MMCTR)")
print("Task: Task2")
print(f"Best Validation AUC (reported on Codabench): {best_auc:.4f}")
print("==========================================")


FINAL OFFICIAL RESULTS
Competition: RS (MicroLens MMCTR)
Task: Task2
Best Validation AUC (reported on Codabench): 0.8995
